# fetch-fonts
This notebook tests converting TTF to vector fonts.

Code source: https://gist.github.com/CatherineH/499a312a04582a00e7559ac0c8f133fa

## download font

In [5]:
!wget https://github.com/adobe-fonts/source-sans/raw/release/TTF/SourceSans3-Regular.ttf

--2023-08-12 20:23:47--  https://github.com/adobe-fonts/source-sans/raw/release/TTF/SourceSans3-Regular.ttf
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/adobe-fonts/source-sans/release/TTF/SourceSans3-Regular.ttf [following]
--2023-08-12 20:23:48--  https://raw.githubusercontent.com/adobe-fonts/source-sans/release/TTF/SourceSans3-Regular.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 431196 (421K) [application/octet-stream]
Saving to: ‘SourceSans3-Regular.ttf’

SourceSans3-Regular 100%[===================>] 421.09K  1.06MB/s    in 0.4s    

2023-08-12 20:23:49 (1.06 MB/s) - ‘SourceSans3-

In [6]:
!ls

fetch-fonts.ipynb  SourceSans3-Regular.ttf  Untitled.ipynb


## convert font

In [2]:
from freetype import Face
from svgpathtools import wsvg, Line, QuadraticBezier, Path

In [3]:
def tuple_to_imag(t):
    return t[0] + t[1] * 1j

In [12]:
face = Face('./SourceSans3-Regular.ttf')
face.set_char_size(48 * 64)
face.load_char('b')
outline = face.glyph.outline
y = [t[1] for t in outline.points]
# flip the points
outline_points = [(p[0], max(y) - p[1]) for p in outline.points]
start, end = 0, 0
paths = []

In [13]:
for i in range(len(outline.contours)):
    end = outline.contours[i]
    points = outline_points[start:end + 1]
    points.append(points[0])
    tags = outline.tags[start:end + 1]
    tags.append(tags[0])

    segments = [[points[0], ], ]
    for j in range(1, len(points)):
        segments[-1].append(points[j])
        if tags[j] and j < (len(points) - 1):
            segments.append([points[j], ])
    for segment in segments:
        if len(segment) == 2:
            paths.append(Line(start=tuple_to_imag(segment[0]),
                              end=tuple_to_imag(segment[1])))
        elif len(segment) == 3:
            paths.append(QuadraticBezier(start=tuple_to_imag(segment[0]),
                                         control=tuple_to_imag(segment[1]),
                                         end=tuple_to_imag(segment[2])))
        elif len(segment) == 4:
            C = ((segment[1][0] + segment[2][0]) / 2.0,
                 (segment[1][1] + segment[2][1]) / 2.0)

            paths.append(QuadraticBezier(start=tuple_to_imag(segment[0]),
                                         control=tuple_to_imag(segment[1]),
                                         end=tuple_to_imag(C)))
            paths.append(QuadraticBezier(start=tuple_to_imag(C),
                                         control=tuple_to_imag(segment[2]),
                                         end=tuple_to_imag(segment[3])))
        else:
            print(f"incompatible segment length: {len(segment)}")
    start = end + 1

In [14]:
path = Path(*paths)

## save font

In [15]:
wsvg(path, filename="SourceSans3-Regular.svg")

In [16]:
!ls

fetch-fonts.ipynb	 SourceSans3-Regular.ttf
SourceSans3-Regular.svg  Untitled.ipynb
